<a href="https://colab.research.google.com/github/ApiLogisticaNoturno/ApiLog-stica4sem/blob/main/Paradas_sem_agregar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

# Caminhos dos arquivos
caminho_2023 = '/content/drive/My Drive/Dados ANTAQ/2023Paradas.csv'
caminho_2024 = '/content/drive/My Drive/Dados ANTAQ/2024Paradas.csv'

# Ler os arquivos com codificação correta para remover BOM e corrigir acentuação
df_2023 = pd.read_csv(caminho_2023, sep=';', encoding='utf-8-sig')
df_2024 = pd.read_csv(caminho_2024, sep=';', encoding='utf-8-sig')

# Adicionar coluna de ano
df_2023['Ano'] = 2023
df_2024['Ano'] = 2024

# Concatenar
df_total = pd.concat([df_2023, df_2024], ignore_index=True)

# Contar linhas
print("Número total de linhas:", df_total.shape[0])

# Converter colunas de data/hora
df_total['DTInicio'] = pd.to_datetime(df_total['DTInicio'], format='%d/%m/%Y %H:%M:%S')
df_total['DTTermino'] = pd.to_datetime(df_total['DTTermino'], format='%d/%m/%Y %H:%M:%S')

# Calcular diferença e converter para horas decimais
df_total['HorasParado'] = (df_total['DTTermino'] - df_total['DTInicio']).dt.total_seconds() / 3600

# Função para converter horas decimais em "XhYmin"
def formatar_horas(horas):
    horas_int = int(horas)
    minutos = int(round((horas - horas_int) * 60))
    return f"{horas_int}h{minutos:02d}min"

# Criar nova coluna formatada
df_total['TempoFormatado'] = df_total['HorasParado'].apply(formatar_horas)

# Exibir para verificar
print(df_total.head())

# (Opcional) Salvar com acentuação preservada
df_total.to_csv('/content/drive/My Drive/Dados ANTAQ/Paradas_2023_2024.csv', sep=';', index=False, encoding='utf-8-sig')


Número total de linhas: 350508
   IDTemposDescontos  IDAtracacao  \
0             940384      1341301   
1             940460      1341486   
2             940488      1341493   
3             940489      1341493   
4             940490      1341493   

                              DescricaoTempoDesconto            DTInicio  \
0                 Preparação de equipamentos do cais 2023-05-26 00:12:00   
1                                Limpeza operacional 2023-05-08 14:00:00   
2  Chuva e/ou outras condições climáticas desfavo... 2023-05-13 20:40:00   
3  Chuva e/ou outras condições climáticas desfavo... 2023-05-13 23:05:00   
4  Chuva e/ou outras condições climáticas desfavo... 2023-05-14 00:00:00   

            DTTermino   Ano  HorasParado TempoFormatado  
0 2023-05-26 00:54:00  2023     0.700000        0h42min  
1 2023-05-08 14:30:00  2023     0.500000        0h30min  
2 2023-05-13 22:10:00  2023     1.500000        1h30min  
3 2023-05-13 23:45:00  2023     0.666667        0h40min  

In [ ]:
from google.colab import drive
import pandas as pd

# Monta o Google Drive (já montado, não precisa remount se já estiver montado)
drive.mount('/content/drive')

# Caminho do arquivo CSV
caminho_csv = '/content/drive/MyDrive/Dados ANTAQ/Base_final_ANTAQ_todas.csv'

# Leitura do CSV (evitando warning com low_memory=False)
df_csv = pd.read_csv(caminho_csv, low_memory=False)

# Limpeza dos nomes de colunas
df_csv.columns = df_csv.columns.str.strip()
df_total.columns = df_total.columns.str.strip()

# Filtra e remove duplicatas pela coluna IDAtracacao
df_csv_filtrado = df_csv[['IDAtracacao', 'Complexo Portuário', 'Terminal', 'Porto Atracação','Mercadoria','Sentido','IDCarga']].drop_duplicates(subset='IDAtracacao')

# Merge mantendo todas as linhas de df_total
df_total_merge = pd.merge(df_total, df_csv_filtrado, on='IDAtracacao', how='left')

# Verificação do número de linhas
print(f"Nº de linhas antes do merge: {len(df_total)}")
print(f"Nº de linhas depois do merge: {len(df_total_merge)}")

# Visualização
print(df_total_merge)

# Caminho onde o arquivo será salvo no Drive
caminho_saida = '/content/drive/MyDrive/Dados ANTAQ/Paradas_2023_2024.csv'

# Salvar o DataFrame em CSV
df_total_merge.to_csv(caminho_saida, index=False, encoding='utf-8-sig')

print("Arquivo salvo com sucesso em:", caminho_saida)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Nº de linhas antes do merge: 350508
Nº de linhas depois do merge: 350508
        IDTemposDescontos  IDAtracacao  \
0                  940384      1341301   
1                  940460      1341486   
2                  940488      1341493   
3                  940489      1341493   
4                  940490      1341493   
...                   ...          ...   
350503            1248243      1520661   
350504            1248244      1520661   
350505            1248245      1520661   
350506            1248246      1520661   
350507            1248247      1520661   

                                   DescricaoTempoDesconto            DTInicio  \
0                      Preparação de equipamentos do cais 2023-05-26 00:12:00   
1                                     Limpeza operacional 2023-05-08 14:00:00   
2       Chuva e/ou outras condições climáticas des

In [ ]:
# 1. Garante que os DataFrames tenham as mesmas colunas antes do merge
colunas_df_total = set(df_total.columns)
colunas_df_merge = set(df_total_merge.columns)

# 2. Mostra as colunas adicionadas no merge
colunas_adicionadas = colunas_df_merge - colunas_df_total
print("Colunas adicionadas no merge:", colunas_adicionadas)

# 3. Cria DataFrames apenas com colunas em comum (para comparação de conteúdo)
df_base = df_total.sort_index()
df_merge_base = df_total_merge[df_total.columns].sort_index()

# 4. Verifica se os dados são exatamente iguais (linha a linha, célula a célula)
iguais = df_base.equals(df_merge_base)
print("Os dados originais e os dados do merge são idênticos (exceto colunas adicionadas)?", iguais)

Colunas adicionadas no merge: {'IDCarga', 'Sentido', 'Complexo Portuário', 'Terminal', 'Porto Atracação', 'Mercadoria'}
Os dados originais e os dados do merge são idênticos (exceto colunas adicionadas)? True


In [ ]:
from google.colab import drive
import pandas as pd()
# Comparar CSV criado com EXCEL (Dados gerais)

# Monta o Google Drive
drive.mount('/content/drive')

# Caminho do Excel
caminho_excel = '/content/drive/MyDrive/Book 2.xlsx'

# Lê o Excel
df_excel = pd.read_excel(caminho_excel)

# Limpa nomes de colunas
df_excel.columns = df_excel.columns.str.strip()
df_csv_filtrado.columns = df_csv_filtrado.columns.str.strip()

# Garante que a coluna IDCarga seja string nos dois
df_excel['IDCarga'] = df_excel['IDCarga'].astype(str)
df_csv_filtrado['IDCarga'] = df_csv_filtrado['IDCarga'].astype(str)

# Faz o merge pelo IDCarga
df_merge = pd.merge(df_excel, df_csv_filtrado, on='IDCarga', how='left')

# Contar quantos IDCarga estão no Excel mas não estão no CSV
nao_encontrados = df_merge[df_merge['IDAtracacao'].isna()]
print(f"IDCarga no Excel mas não encontrados no CSV: {len(nao_encontrados)}")

print(nao_encontrados['IDCarga'].unique())





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
IDCarga no Excel mas não encontrados no CSV: 653
['36545711' '36545701' '36545713' '36545693' '36545695' '36545697'
 '36545715' '36545719' '36545691' '36545709' '36545717' '36545687'
 '36545699' '36545683' '36545681' '36545707' '36545685' '36545689'
 '36361818' '36361814' '36545705' '36545703' '36445110' '36445109'
 '36445108' '36445107' '36445104' '36445103' '36445102' '36445101'
 '36445100' '36445099' '36445098' '36445097' '36445096' '36445095'
 '36445094' '37009068' '37009067' '37009066' '37009065' '37009064'
 '37009063' '37009062' '37009061' '36840584' '36840583' '36840582'
 '36840581' '36840580' '36840579' '36840578' '36215223' '36215222'
 '36215221' '36215220' '36215219' '36215218' '36215217' '36215216'
 '36215215' '36215214' '36215213' '36840794' '36840792' '36840791'
 '36840790' '36840787' '36840786' '36840785' '36840784' '36840782'
 '36840781' '36840